# Odor Following Navigation Algorithms

## Interactive Demonstration of Gradient-Following Strategies

This notebook provides an educational demonstration of gradient-following algorithms used in odor plume navigation. You'll learn about the fundamental concepts behind how autonomous agents can locate odor sources using local gradient information and simple decision-making strategies.

### Learning Objectives

1. **Understand gradient-following algorithms** - Learn how agents detect and follow odor gradients
2. **Explore direction selection strategies** - Analyze different approaches to choosing movement directions
3. **Investigate adaptive speed control** - See how agents adjust their speed based on odor concentration
4. **Parameter sensitivity analysis** - Experiment with algorithm parameters to understand their effects
5. **Visualize decision-making processes** - Observe real-time agent decision making through interactive plots

### Algorithm Overview

The gradient-following algorithm implemented here uses a simple but effective strategy:

1. **Direction Testing**: The agent tests multiple directions around its current position
2. **Gradient Detection**: It compares odor concentrations at these test positions
3. **Direction Selection**: It chooses the direction with the highest odor concentration
4. **Adaptive Speed**: Speed is adjusted based on the strength of the detected gradient
5. **Movement Execution**: The agent moves in the selected direction with the calculated speed

This approach mimics the behavior of many biological organisms that navigate using chemical gradients.

## Setup and Configuration

First, let's import the necessary libraries and configure our simulation environment.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.widgets import Slider, Button
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for notebook display
%matplotlib inline
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

In [ ]:
# Import our new project modules
try:
    # Import core navigation components
    from {{cookiecutter.project_slug}}.core.navigator import NavigatorFactory, NavigatorProtocol
    
    # Import visualization utilities
    from {{cookiecutter.project_slug}}.utils.visualization import SimulationVisualization, visualize_trajectory
    
    # Import seed manager for reproducibility
    from {{cookiecutter.project_slug}}.utils.seed_manager import set_global_seed, seed_context
    
    # Import configuration schemas
    from {{cookiecutter.project_slug}}.config.schemas import NavigatorConfig, SingleAgentConfig
    
    print("✅ Successfully imported all project modules")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure the project is properly installed and accessible")
    raise

## Reproducible Experiment Setup

For reproducible research, we'll use the seed manager to ensure consistent results across different runs.

In [ ]:
# Set up reproducible random state
EXPERIMENT_SEED = 42
EXPERIMENT_ID = "odor_following_demo"

# Initialize global seed manager for reproducibility
seed_manager = set_global_seed(EXPERIMENT_SEED, experiment_id=EXPERIMENT_ID)

print(f"🎯 Experiment initialized with seed: {EXPERIMENT_SEED}")
print(f"📊 Experiment ID: {EXPERIMENT_ID}")
print(f"⏱️ Initialization time: {seed_manager._initialization_time:.2f}ms")

# Display reproducibility information
repro_info = seed_manager.get_reproducibility_info()
print(f"🖥️ Platform: {repro_info['platform_info']['platform']}")
print(f"🐍 Python: {'.'.join(map(str, repro_info['platform_info']['python_version']))}")
print(f"🔢 NumPy: {repro_info['platform_info']['numpy_version']}")

## Odor Environment Creation

Let's create a Gaussian odor field that represents a realistic odor plume environment. This environment will have multiple odor sources with different intensities and spatial distributions.

In [ ]:
def create_gaussian_odor_field(width: int = 50, height: int = 50, centers=None):
    """
    Create a Gaussian odor field with multiple sources.
    
    This function generates a 2D odor concentration field using Gaussian distributions
    to represent odor sources. Multiple sources can be specified with different
    intensities and spatial spreads to create realistic odor landscapes.
    
    Args:
        width: Width of the environment grid
        height: Height of the environment grid  
        centers: List of (x, y, intensity, sigma) tuples defining odor sources
    
    Returns:
        2D numpy array with normalized odor concentrations [0, 1]
    """
    if centers is None:
        # Default: two odor sources with different characteristics
        centers = [
            (width * 0.7, height * 0.6, 1.0, 5.0),   # Primary source: (x, y, intensity, sigma)
            (width * 0.3, height * 0.3, 0.7, 7.0),   # Secondary source: broader, less intense
        ]
    
    # Create coordinate meshgrid for vectorized computation
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    odor_field = np.zeros((height, width), dtype=np.float32)
    
    # Add each Gaussian odor source to the field
    for cx, cy, intensity, sigma in centers:
        # Calculate Gaussian distribution: exp(-r²/2σ²)
        gaussian = np.exp(-((x-cx)**2 + (y-cy)**2) / (2*sigma**2))
        odor_field += intensity * gaussian
    
    # Normalize to [0, 1] range for consistent interpretation
    max_val = np.max(odor_field)
    if max_val > 1:
        odor_field /= max_val
        
    return odor_field

# Create the odor environment
ENV_WIDTH, ENV_HEIGHT = 50, 50
odor_field = create_gaussian_odor_field(ENV_WIDTH, ENV_HEIGHT)

# Visualize the odor field
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: 2D heatmap
im1 = ax1.imshow(odor_field, origin='lower', cmap='viridis', extent=[0, ENV_WIDTH, 0, ENV_HEIGHT])
ax1.set_title('Odor Concentration Field (2D View)', fontsize=14, fontweight='bold')
ax1.set_xlabel('X Position')
ax1.set_ylabel('Y Position')
plt.colorbar(im1, ax=ax1, label='Odor Concentration')

# Plot 2: 3D surface plot for better understanding of gradients
x_3d, y_3d = np.meshgrid(np.arange(ENV_WIDTH), np.arange(ENV_HEIGHT))
ax2 = fig.add_subplot(122, projection='3d')
surf = ax2.plot_surface(x_3d, y_3d, odor_field, cmap='viridis', alpha=0.8)
ax2.set_title('Odor Concentration Field (3D View)', fontsize=14, fontweight='bold')
ax2.set_xlabel('X Position')
ax2.set_ylabel('Y Position')
ax2.set_zlabel('Odor Concentration')

plt.tight_layout()
plt.show()

print(f"🌊 Created odor field: {ENV_WIDTH}×{ENV_HEIGHT} grid")
print(f"📊 Concentration range: [{np.min(odor_field):.3f}, {np.max(odor_field):.3f}]")
print(f"🎯 Peak locations at: {np.unravel_index(np.argmax(odor_field), odor_field.shape)}")

## Agent Configuration and Initialization

Now let's create and configure our navigation agent using the new project structure. We'll use the NavigatorFactory to create a single agent with specific starting parameters.

In [ ]:
# Configure the navigator using the new configuration system
navigator_config = SingleAgentConfig(
    position=(5, 5),        # Start at bottom-left corner
    orientation=0.0,        # Initially facing right (0 degrees)
    speed=0.0,              # Start with zero speed
    max_speed=1.0,          # Maximum speed constraint
    angular_velocity=0.0    # No initial rotation
)

# Create navigator using the factory method
navigator = NavigatorFactory.from_config(navigator_config)

print(f"🤖 Navigator created successfully")
print(f"📍 Starting position: {navigator.positions[0]}")
print(f"🧭 Starting orientation: {navigator.orientations[0]:.1f}°")
print(f"🚀 Max speed: {navigator.max_speeds[0]:.1f} units/step")
print(f"👥 Number of agents: {navigator.num_agents}")

## Interactive Parameter Control

Let's create interactive widgets to control the algorithm parameters in real-time. This allows you to experiment with different settings and understand their effects on agent behavior.

In [ ]:
# Create interactive parameter controls
class AlgorithmParameters:
    """Container for algorithm parameters with interactive controls."""
    
    def __init__(self):
        # Core algorithm parameters
        self.test_directions = [0, 45, 90, 135, 180, 225, 270, 315]  # Directions to test
        self.test_distance = 2.0      # How far ahead to test for odor
        self.base_speed = 0.3         # Minimum movement speed
        self.speed_multiplier = 2.0   # How much gradient affects speed
        self.convergence_threshold = 0.9  # Odor level to consider "arrived"
        
        # Create interactive widgets
        self.create_widgets()
    
    def create_widgets(self):
        """Create interactive parameter control widgets."""
        
        # Direction testing parameters
        self.directions_widget = widgets.SelectMultiple(
            options=[0, 45, 90, 135, 180, 225, 270, 315],
            value=[0, 45, 90, 135, 180, 225, 270, 315],
            description='Test Directions:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', height='120px')
        )
        
        # Test distance slider
        self.distance_widget = widgets.FloatSlider(
            value=2.0,
            min=0.5,
            max=5.0,
            step=0.1,
            description='Test Distance:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # Base speed slider
        self.base_speed_widget = widgets.FloatSlider(
            value=0.3,
            min=0.1,
            max=1.0,
            step=0.05,
            description='Base Speed:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # Speed multiplier slider
        self.speed_mult_widget = widgets.FloatSlider(
            value=2.0,
            min=0.5,
            max=5.0,
            step=0.1,
            description='Speed Multiplier:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # Convergence threshold slider
        self.threshold_widget = widgets.FloatSlider(
            value=0.9,
            min=0.5,
            max=1.0,
            step=0.01,
            description='Convergence Threshold:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # Control buttons
        self.reset_button = widgets.Button(
            description='Reset Parameters',
            button_style='info',
            layout=widgets.Layout(width='150px')
        )
        
        self.random_button = widgets.Button(
            description='Randomize',
            button_style='warning',
            layout=widgets.Layout(width='150px')
        )
        
        # Set up event handlers
        self.reset_button.on_click(self.reset_parameters)
        self.random_button.on_click(self.randomize_parameters)
    
    def update_from_widgets(self):
        """Update parameters from widget values."""
        self.test_directions = list(self.directions_widget.value)
        self.test_distance = self.distance_widget.value
        self.base_speed = self.base_speed_widget.value
        self.speed_multiplier = self.speed_mult_widget.value
        self.convergence_threshold = self.threshold_widget.value
    
    def reset_parameters(self, button):
        """Reset parameters to default values."""
        self.directions_widget.value = [0, 45, 90, 135, 180, 225, 270, 315]
        self.distance_widget.value = 2.0
        self.base_speed_widget.value = 0.3
        self.speed_mult_widget.value = 2.0
        self.threshold_widget.value = 0.9
    
    def randomize_parameters(self, button):
        """Randomize parameters within reasonable ranges."""
        # Randomly select 4-8 directions
        all_dirs = [0, 45, 90, 135, 180, 225, 270, 315]
        n_dirs = np.random.randint(4, 9)
        selected_dirs = sorted(np.random.choice(all_dirs, n_dirs, replace=False))
        self.directions_widget.value = selected_dirs
        
        # Randomize other parameters
        self.distance_widget.value = np.random.uniform(1.0, 4.0)
        self.base_speed_widget.value = np.random.uniform(0.1, 0.8)
        self.speed_mult_widget.value = np.random.uniform(1.0, 4.0)
        self.threshold_widget.value = np.random.uniform(0.7, 0.95)
    
    def display_controls(self):
        """Display the parameter control widgets."""
        print("🎛️ Algorithm Parameter Controls")
        print("Adjust these parameters to see how they affect the agent's behavior:")
        print()
        
        # Group widgets for better layout
        direction_box = widgets.VBox([
            widgets.HTML("<b>Direction Testing:</b>"),
            self.directions_widget,
            widgets.HTML("<i>Hold Ctrl to select multiple directions</i>")
        ])
        
        parameter_box = widgets.VBox([
            widgets.HTML("<b>Movement Parameters:</b>"),
            self.distance_widget,
            self.base_speed_widget,
            self.speed_mult_widget,
            self.threshold_widget
        ])
        
        button_box = widgets.HBox([
            self.reset_button,
            self.random_button
        ])
        
        control_box = widgets.HBox([
            direction_box,
            parameter_box
        ])
        
        display(widgets.VBox([
            control_box,
            button_box
        ]))

# Create parameter controller
params = AlgorithmParameters()
params.display_controls()

## Gradient-Following Algorithm Implementation

Now let's implement the core gradient-following algorithm. This function represents the "brain" of our agent, making decisions about where to move based on local odor information.

In [ ]:
def gradient_following_step(navigator: NavigatorProtocol, odor_field: np.ndarray, 
                          params: AlgorithmParameters, debug: bool = False):
    """
    Execute one step of the gradient-following algorithm.
    
    This function implements the core logic of odor source seeking:
    1. Sample current odor concentration
    2. Test multiple directions for better odor concentrations
    3. Select the direction with highest concentration
    4. Calculate adaptive speed based on gradient strength
    5. Update navigator orientation and speed
    
    Args:
        navigator: Navigator instance implementing NavigatorProtocol
        odor_field: 2D array of odor concentrations
        params: Algorithm parameters object
        debug: Whether to return debug information
    
    Returns:
        If debug=True: tuple of (decision_info, visualization_data)
        If debug=False: None
    """
    # Update parameters from widgets
    params.update_from_widgets()
    
    # Get current state
    current_pos = navigator.positions[0]  # Single agent
    current_odor = navigator.sample_odor(odor_field)
    
    # Initialize tracking variables
    best_direction = navigator.orientations[0]  # Default to current orientation
    best_odor = current_odor
    test_results = []
    
    # Test each direction to find the best odor concentration
    for test_direction in params.test_directions:
        # Calculate test position
        test_radians = np.radians(test_direction)
        test_x = current_pos[0] + np.cos(test_radians) * params.test_distance
        test_y = current_pos[1] + np.sin(test_radians) * params.test_distance
        
        # Create temporary navigator to sample odor at test position
        test_navigator = NavigatorFactory.single_agent(position=(test_x, test_y))
        test_odor = test_navigator.sample_odor(odor_field)
        
        # Store test result for debugging/visualization
        test_results.append({
            'direction': test_direction,
            'position': (test_x, test_y),
            'odor': test_odor,
            'improvement': test_odor - current_odor
        })
        
        # Update best direction if this test is better
        if test_odor > best_odor:
            best_odor = test_odor
            best_direction = test_direction
    
    # Calculate speed based on gradient strength
    odor_improvement = max(0, best_odor - current_odor)
    calculated_speed = params.base_speed + odor_improvement * params.speed_multiplier
    
    # Apply speed constraints
    final_speed = min(calculated_speed, navigator.max_speeds[0])
    
    # Update navigator orientation and speed
    navigator.reset(orientation=best_direction, speed=final_speed)
    
    # Prepare debug information if requested
    if debug:
        decision_info = {
            'current_position': current_pos,
            'current_odor': current_odor,
            'best_direction': best_direction,
            'best_odor': best_odor,
            'odor_improvement': odor_improvement,
            'calculated_speed': calculated_speed,
            'final_speed': final_speed,
            'test_results': test_results
        }
        
        # Sort test results by odor level for easier analysis
        test_results_sorted = sorted(test_results, key=lambda x: x['odor'], reverse=True)
        
        visualization_data = {
            'test_positions': [r['position'] for r in test_results],
            'test_odors': [r['odor'] for r in test_results],
            'test_directions': [r['direction'] for r in test_results],
            'best_test_idx': test_results.index(max(test_results, key=lambda x: x['odor']))
        }
        
        return decision_info, visualization_data

print("🧠 Gradient-following algorithm implementation ready")
print("📊 Algorithm features:")
print("   • Multi-directional odor testing")
print("   • Adaptive speed control based on gradient strength")
print("   • Configurable direction sets and parameters")
print("   • Debug mode for decision analysis")

## Step-by-Step Algorithm Visualization

Let's create a detailed visualization that shows exactly what the agent is "thinking" at each step. This helps understand the decision-making process.

In [ ]:
def visualize_decision_process(navigator: NavigatorProtocol, odor_field: np.ndarray, 
                             params: AlgorithmParameters, step_num: int = 0):
    """
    Create a detailed visualization of the agent's decision-making process.
    
    This function creates a comprehensive plot showing:
    - The odor field background
    - Agent's current position and orientation
    - Test positions and their odor readings
    - The selected best direction
    - A decision analysis panel
    """
    # Execute algorithm step with debug information
    decision_info, viz_data = gradient_following_step(navigator, odor_field, params, debug=True)
    
    # Create figure with subplots
    fig = plt.figure(figsize=(16, 10))
    
    # Main visualization panel
    ax_main = plt.subplot2grid((3, 3), (0, 0), rowspan=3, colspan=2)
    
    # Decision analysis panels
    ax_directions = plt.subplot2grid((3, 3), (0, 2))
    ax_speeds = plt.subplot2grid((3, 3), (1, 2))
    ax_info = plt.subplot2grid((3, 3), (2, 2))
    
    # Plot odor field as background
    im = ax_main.imshow(odor_field, origin='lower', cmap='viridis', alpha=0.7,
                       extent=[0, odor_field.shape[1], 0, odor_field.shape[0]])
    
    # Plot agent's current position
    agent_pos = decision_info['current_position']
    ax_main.scatter(agent_pos[0], agent_pos[1], c='red', s=200, marker='*', 
                   edgecolors='white', linewidth=2, label='Agent', zorder=10)
    
    # Plot test positions with color-coded odor levels
    test_positions = viz_data['test_positions']
    test_odors = viz_data['test_odors']
    test_dirs = viz_data['test_directions']
    best_idx = viz_data['best_test_idx']
    
    # Create color map for test positions
    test_colors = plt.cm.plasma(np.array(test_odors) / max(test_odors) if max(test_odors) > 0 else [0.5] * len(test_odors))
    
    for i, (pos, odor, direction) in enumerate(zip(test_positions, test_odors, test_dirs)):
        # Different marker for best direction
        if i == best_idx:
            ax_main.scatter(pos[0], pos[1], c='lime', s=150, marker='D', 
                           edgecolors='darkgreen', linewidth=2, label='Best Direction', zorder=9)
        else:
            ax_main.scatter(pos[0], pos[1], c=[test_colors[i]], s=100, marker='o', 
                           edgecolors='white', linewidth=1, alpha=0.8, zorder=8)
        
        # Add direction labels
        ax_main.annotate(f'{direction}°\n{odor:.3f}', pos, 
                        xytext=(5, 5), textcoords='offset points',
                        fontsize=8, ha='left', va='bottom',
                        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
    
    # Draw movement vector for selected direction
    best_dir_rad = np.radians(decision_info['best_direction'])
    arrow_length = decision_info['final_speed'] * 3  # Scale for visibility
    dx = arrow_length * np.cos(best_dir_rad)
    dy = arrow_length * np.sin(best_dir_rad)
    
    ax_main.arrow(agent_pos[0], agent_pos[1], dx, dy,
                 head_width=1, head_length=1, fc='red', ec='red', 
                 linewidth=3, alpha=0.8, zorder=11)
    
    # Configure main plot
    ax_main.set_title(f'Step {step_num}: Agent Decision Process', fontsize=14, fontweight='bold')
    ax_main.set_xlabel('X Position')
    ax_main.set_ylabel('Y Position')
    ax_main.legend(loc='upper left')
    ax_main.grid(True, alpha=0.3)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax_main, shrink=0.8)
    cbar.set_label('Odor Concentration')
    
    # Direction analysis plot
    directions = [r['direction'] for r in decision_info['test_results']]
    odor_values = [r['odor'] for r in decision_info['test_results']]
    colors = ['lime' if d == decision_info['best_direction'] else 'skyblue' for d in directions]
    
    bars = ax_directions.bar(range(len(directions)), odor_values, color=colors, alpha=0.7)
    ax_directions.set_xticks(range(len(directions)))
    ax_directions.set_xticklabels([f"{d}°" for d in directions], rotation=45)
    ax_directions.set_title('Direction Testing Results', fontweight='bold')
    ax_directions.set_ylabel('Odor Level')
    ax_directions.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, odor_values):
        height = bar.get_height()
        ax_directions.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                          f'{value:.3f}', ha='center', va='bottom', fontsize=8)
    
    # Speed calculation visualization
    speed_components = [
        ('Base Speed', params.base_speed, 'lightblue'),
        ('Gradient Bonus', decision_info['odor_improvement'] * params.speed_multiplier, 'orange'),
        ('Final Speed', decision_info['final_speed'], 'red')
    ]
    
    speed_names = [sc[0] for sc in speed_components]
    speed_values = [sc[1] for sc in speed_components]
    speed_colors = [sc[2] for sc in speed_components]
    
    bars_speed = ax_speeds.bar(speed_names, speed_values, color=speed_colors, alpha=0.7)
    ax_speeds.set_title('Speed Calculation', fontweight='bold')
    ax_speeds.set_ylabel('Speed (units/step)')
    ax_speeds.tick_params(axis='x', rotation=45)
    ax_speeds.grid(True, alpha=0.3)
    
    # Add value labels on speed bars
    for bar, value in zip(bars_speed, speed_values):
        height = bar.get_height()
        ax_speeds.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                      f'{value:.3f}', ha='center', va='bottom', fontsize=9)
    
    # Information panel
    ax_info.axis('off')
    info_text = f"""DECISION SUMMARY
━━━━━━━━━━━━━━━━━━━━
Current Position: ({agent_pos[0]:.1f}, {agent_pos[1]:.1f})
Current Odor: {decision_info['current_odor']:.4f}

Best Direction: {decision_info['best_direction']:.0f}°
Best Odor Found: {decision_info['best_odor']:.4f}
Improvement: +{decision_info['odor_improvement']:.4f}

Calculated Speed: {decision_info['calculated_speed']:.3f}
Final Speed: {decision_info['final_speed']:.3f}
Speed Limited: {'Yes' if decision_info['calculated_speed'] != decision_info['final_speed'] else 'No'}

Directions Tested: {len(params.test_directions)}
Test Distance: {params.test_distance:.1f} units"""
    
    ax_info.text(0.05, 0.95, info_text, transform=ax_info.transAxes, 
                fontsize=10, verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    return decision_info

print("🔍 Decision process visualization ready")
print("This will show the agent's 'thought process' step by step")

## Interactive Single-Step Analysis

Let's run a single step of the algorithm and analyze the decision-making process in detail. You can adjust the parameters above and re-run this cell to see how different settings affect the agent's behavior.

In [ ]:
# Reset navigator to starting position for single-step analysis
with seed_context(EXPERIMENT_SEED):
    navigator.reset(
        position=(5, 5),
        orientation=0.0,
        speed=0.0
    )

    print("🎯 Single-Step Analysis")
    print("Adjust the parameters above and run this cell to see the effects!")
    print()
    
    # Visualize the decision process
    decision_info = visualize_decision_process(navigator, odor_field, params, step_num=1)
    
    # Print detailed analysis
    print("\n📊 Detailed Analysis:")
    print(f"Current odor concentration: {decision_info['current_odor']:.4f}")
    print(f"Number of directions tested: {len(decision_info['test_results'])}")
    
    # Analyze test results
    test_results = sorted(decision_info['test_results'], key=lambda x: x['odor'], reverse=True)
    print("\nTop 3 directions by odor concentration:")
    for i, result in enumerate(test_results[:3]):
        symbol = "🥇" if i == 0 else "🥈" if i == 1 else "🥉"
        print(f"  {symbol} {result['direction']:3.0f}°: {result['odor']:.4f} (improvement: {result['improvement']:+.4f})")
    
    # Speed analysis
    print(f"\n🚀 Speed Analysis:")
    print(f"  Base speed: {params.base_speed:.3f}")
    print(f"  Gradient bonus: {decision_info['odor_improvement'] * params.speed_multiplier:.3f}")
    print(f"  Total calculated: {decision_info['calculated_speed']:.3f}")
    print(f"  Final speed (after limits): {decision_info['final_speed']:.3f}")
    
    if decision_info['odor_improvement'] == 0:
        print("⚠️ No improvement found - agent may be at local optimum or need different test directions")

## Complete Navigation Simulation

Now let's run a complete simulation showing how the agent navigates from its starting position to the odor source. This demonstrates the full algorithm in action.

In [ ]:
def run_complete_simulation(max_steps: int = 40, show_animation: bool = True):
    """
    Run a complete odor-following simulation with visualization.
    
    Args:
        max_steps: Maximum number of simulation steps
        show_animation: Whether to show the animated visualization
    
    Returns:
        Dictionary containing simulation results and trajectory data
    """
    # Reset navigator to starting position
    with seed_context(EXPERIMENT_SEED):
        navigator.reset(
            position=(5, 5),
            orientation=0.0,
            speed=0.0
        )
        
        # Initialize tracking variables
        trajectory = []
        orientations = []
        speeds = []
        odor_readings = []
        decision_log = []
        dt = 0.5  # Time step
        
        # Get current parameters
        params.update_from_widgets()
        
        print(f"🚀 Starting simulation with parameters:")
        print(f"   Test directions: {params.test_directions}")
        print(f"   Test distance: {params.test_distance:.1f}")
        print(f"   Base speed: {params.base_speed:.2f}")
        print(f"   Speed multiplier: {params.speed_multiplier:.1f}")
        print(f"   Convergence threshold: {params.convergence_threshold:.2f}")
        print()
        
        # Run simulation steps
        for step in range(max_steps):
            # Record current state
            current_pos = navigator.positions[0].copy()
            current_orientation = navigator.orientations[0]
            current_speed = navigator.speeds[0]
            current_odor = navigator.sample_odor(odor_field)
            
            trajectory.append(current_pos)
            orientations.append(current_orientation)
            speeds.append(current_speed)
            odor_readings.append(current_odor)
            
            # Execute gradient-following algorithm
            decision_info, _ = gradient_following_step(navigator, odor_field, params, debug=True)
            decision_log.append(decision_info)
            
            # Update navigator position
            navigator.step(odor_field, dt)
            
            # Check convergence
            if current_odor >= params.convergence_threshold:
                print(f"🎯 Convergence achieved at step {step + 1}!")
                print(f"   Final position: ({current_pos[0]:.1f}, {current_pos[1]:.1f})")
                print(f"   Final odor level: {current_odor:.4f}")
                break
                
            # Progress update
            if (step + 1) % 10 == 0:
                print(f"Step {step + 1:2d}: pos=({current_pos[0]:5.1f},{current_pos[1]:5.1f}), "
                      f"odor={current_odor:.4f}, speed={current_speed:.3f}")
        
        # Convert lists to arrays for easier analysis
        trajectory = np.array(trajectory)
        orientations = np.array(orientations)
        speeds = np.array(speeds)
        odor_readings = np.array(odor_readings)
        
        # Create visualization if requested
        if show_animation:
            # Create static trajectory plot
            fig, axes = plt.subplots(2, 2, figsize=(16, 12))
            
            # Plot 1: Trajectory on odor field
            ax1 = axes[0, 0]
            im = ax1.imshow(odor_field, origin='lower', cmap='viridis', alpha=0.7,
                           extent=[0, odor_field.shape[1], 0, odor_field.shape[0]])
            ax1.plot(trajectory[:, 0], trajectory[:, 1], 'r-', linewidth=2, alpha=0.8, label='Trajectory')
            ax1.scatter(trajectory[0, 0], trajectory[0, 1], c='lime', s=150, marker='o', 
                       edgecolors='black', linewidth=2, label='Start', zorder=10)
            ax1.scatter(trajectory[-1, 0], trajectory[-1, 1], c='red', s=150, marker='*', 
                       edgecolors='white', linewidth=2, label='End', zorder=10)
            ax1.set_title('Complete Navigation Trajectory', fontweight='bold')
            ax1.set_xlabel('X Position')
            ax1.set_ylabel('Y Position')
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            plt.colorbar(im, ax=ax1, label='Odor Concentration')
            
            # Plot 2: Odor concentration over time
            ax2 = axes[0, 1]
            ax2.plot(odor_readings, 'b-', linewidth=2, marker='o', markersize=4)
            ax2.axhline(y=params.convergence_threshold, color='red', linestyle='--', 
                       label=f'Convergence Threshold ({params.convergence_threshold:.2f})')
            ax2.set_title('Odor Concentration Over Time', fontweight='bold')
            ax2.set_xlabel('Step Number')
            ax2.set_ylabel('Odor Concentration')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            
            # Plot 3: Speed over time
            ax3 = axes[1, 0]
            ax3.plot(speeds, 'g-', linewidth=2, marker='s', markersize=4)
            ax3.axhline(y=params.base_speed, color='orange', linestyle='--', 
                       label=f'Base Speed ({params.base_speed:.2f})')
            ax3.axhline(y=navigator.max_speeds[0], color='red', linestyle='--', 
                       label=f'Max Speed ({navigator.max_speeds[0]:.2f})')
            ax3.set_title('Agent Speed Over Time', fontweight='bold')
            ax3.set_xlabel('Step Number')
            ax3.set_ylabel('Speed (units/step)')
            ax3.legend()
            ax3.grid(True, alpha=0.3)
            
            # Plot 4: Orientation over time
            ax4 = axes[1, 1]
            ax4.plot(orientations, 'm-', linewidth=2, marker='^', markersize=4)
            ax4.set_title('Agent Orientation Over Time', fontweight='bold')
            ax4.set_xlabel('Step Number')
            ax4.set_ylabel('Orientation (degrees)')
            ax4.set_ylim(0, 360)
            ax4.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
        
        # Calculate performance metrics
        total_distance = np.sum(np.linalg.norm(np.diff(trajectory, axis=0), axis=1))
        final_odor = odor_readings[-1]
        steps_taken = len(trajectory)
        success = final_odor >= params.convergence_threshold
        
        # Compile results
        results = {
            'trajectory': trajectory,
            'orientations': orientations,
            'speeds': speeds,
            'odor_readings': odor_readings,
            'decision_log': decision_log,
            'parameters': {
                'test_directions': params.test_directions.copy(),
                'test_distance': params.test_distance,
                'base_speed': params.base_speed,
                'speed_multiplier': params.speed_multiplier,
                'convergence_threshold': params.convergence_threshold
            },
            'metrics': {
                'total_distance': total_distance,
                'final_odor': final_odor,
                'steps_taken': steps_taken,
                'success': success,
                'efficiency': final_odor / total_distance if total_distance > 0 else 0
            }
        }
        
        return results

# Run the complete simulation
print("🎬 Running Complete Odor Following Simulation")
print("" + "="*50)

simulation_results = run_complete_simulation(max_steps=40, show_animation=True)

# Display performance summary
metrics = simulation_results['metrics']
print("\n📈 Simulation Performance Summary:")
print("" + "-"*40)
print(f"Success: {'✅ Yes' if metrics['success'] else '❌ No'}")
print(f"Steps taken: {metrics['steps_taken']}")
print(f"Total distance traveled: {metrics['total_distance']:.2f} units")
print(f"Final odor concentration: {metrics['final_odor']:.4f}")
print(f"Navigation efficiency: {metrics['efficiency']:.4f}")

## Parameter Sensitivity Analysis

Let's systematically explore how different parameter settings affect the agent's performance. This helps understand which parameters are most critical for successful navigation.

In [ ]:
def parameter_sensitivity_analysis():
    """
    Perform systematic parameter sensitivity analysis.
    
    This function tests different parameter combinations to understand
    their effects on navigation performance.
    """
    print("🔬 Parameter Sensitivity Analysis")
    print("Testing different parameter combinations...")
    
    # Define parameter ranges to test
    test_cases = [
        {
            'name': 'Baseline (8 directions)',
            'directions': [0, 45, 90, 135, 180, 225, 270, 315],
            'test_distance': 2.0,
            'base_speed': 0.3,
            'speed_multiplier': 2.0
        },
        {
            'name': 'Cardinal Only (4 directions)',
            'directions': [0, 90, 180, 270],
            'test_distance': 2.0,
            'base_speed': 0.3,
            'speed_multiplier': 2.0
        },
        {
            'name': 'High Resolution (16 directions)',
            'directions': [i * 22.5 for i in range(16)],
            'test_distance': 2.0,
            'base_speed': 0.3,
            'speed_multiplier': 2.0
        },
        {
            'name': 'Short Range Sensing',
            'directions': [0, 45, 90, 135, 180, 225, 270, 315],
            'test_distance': 1.0,
            'base_speed': 0.3,
            'speed_multiplier': 2.0
        },
        {
            'name': 'Long Range Sensing',
            'directions': [0, 45, 90, 135, 180, 225, 270, 315],
            'test_distance': 4.0,
            'base_speed': 0.3,
            'speed_multiplier': 2.0
        },
        {
            'name': 'Conservative Speed',
            'directions': [0, 45, 90, 135, 180, 225, 270, 315],
            'test_distance': 2.0,
            'base_speed': 0.1,
            'speed_multiplier': 1.0
        },
        {
            'name': 'Aggressive Speed',
            'directions': [0, 45, 90, 135, 180, 225, 270, 315],
            'test_distance': 2.0,
            'base_speed': 0.5,
            'speed_multiplier': 4.0
        }
    ]
    
    results = []
    
    for i, test_case in enumerate(test_cases):
        print(f"\n🧪 Test {i+1}/{len(test_cases)}: {test_case['name']}")
        
        # Set up parameters for this test
        params.directions_widget.value = test_case['directions']
        params.distance_widget.value = test_case['test_distance']
        params.base_speed_widget.value = test_case['base_speed']
        params.speed_mult_widget.value = test_case['speed_multiplier']
        params.threshold_widget.value = 0.9  # Keep convergence threshold constant
        
        # Run simulation (without visualization for speed)
        with seed_context(EXPERIMENT_SEED):  # Ensure consistent starting conditions
            test_results = run_complete_simulation(max_steps=50, show_animation=False)
        
        # Store results
        test_result = {
            'name': test_case['name'],
            'parameters': test_case,
            'metrics': test_results['metrics'],
            'trajectory_length': len(test_results['trajectory'])
        }
        results.append(test_result)
        
        # Print summary
        m = test_results['metrics']
        print(f"   Result: {'✅ Success' if m['success'] else '❌ Failed'} "
              f"in {m['steps_taken']} steps, "
              f"final odor: {m['final_odor']:.4f}, "
              f"efficiency: {m['efficiency']:.4f}")
    
    # Create comparison visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Extract data for plotting
    names = [r['name'] for r in results]
    steps = [r['metrics']['steps_taken'] for r in results]
    distances = [r['metrics']['total_distance'] for r in results]
    final_odors = [r['metrics']['final_odor'] for r in results]
    efficiencies = [r['metrics']['efficiency'] for r in results]
    successes = [r['metrics']['success'] for r in results]
    
    # Color code by success
    colors = ['green' if s else 'red' for s in successes]
    
    # Plot 1: Steps taken
    ax1 = axes[0, 0]
    bars1 = ax1.bar(range(len(names)), steps, color=colors, alpha=0.7)
    ax1.set_title('Steps to Completion/Timeout', fontweight='bold')
    ax1.set_ylabel('Number of Steps')
    ax1.set_xticks(range(len(names)))
    ax1.set_xticklabels(names, rotation=45, ha='right')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Total distance
    ax2 = axes[0, 1]
    bars2 = ax2.bar(range(len(names)), distances, color=colors, alpha=0.7)
    ax2.set_title('Total Distance Traveled', fontweight='bold')
    ax2.set_ylabel('Distance (units)')
    ax2.set_xticks(range(len(names)))
    ax2.set_xticklabels(names, rotation=45, ha='right')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Final odor concentration
    ax3 = axes[1, 0]
    bars3 = ax3.bar(range(len(names)), final_odors, color=colors, alpha=0.7)
    ax3.axhline(y=0.9, color='black', linestyle='--', label='Convergence Threshold')
    ax3.set_title('Final Odor Concentration', fontweight='bold')
    ax3.set_ylabel('Odor Concentration')
    ax3.set_xticks(range(len(names)))
    ax3.set_xticklabels(names, rotation=45, ha='right')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Navigation efficiency
    ax4 = axes[1, 1]
    bars4 = ax4.bar(range(len(names)), efficiencies, color=colors, alpha=0.7)
    ax4.set_title('Navigation Efficiency (Odor/Distance)', fontweight='bold')
    ax4.set_ylabel('Efficiency')
    ax4.set_xticks(range(len(names)))
    ax4.set_xticklabels(names, rotation=45, ha='right')
    ax4.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for ax, values in [(ax1, steps), (ax2, distances), (ax3, final_odors), (ax4, efficiencies)]:
        bars = ax.patches
        for bar, value in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{value:.2f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Print analysis summary
    print("\n📊 Analysis Summary:")
    print("" + "="*50)
    
    successful = [r for r in results if r['metrics']['success']]
    failed = [r for r in results if not r['metrics']['success']]
    
    print(f"Successful strategies: {len(successful)}/{len(results)}")
    
    if successful:
        best_efficiency = max(successful, key=lambda x: x['metrics']['efficiency'])
        fastest = min(successful, key=lambda x: x['metrics']['steps_taken'])
        
        print(f"\n🏆 Best efficiency: {best_efficiency['name']}")
        print(f"   Efficiency: {best_efficiency['metrics']['efficiency']:.4f}")
        print(f"   Steps: {best_efficiency['metrics']['steps_taken']}")
        
        print(f"\n⚡ Fastest completion: {fastest['name']}")
        print(f"   Steps: {fastest['metrics']['steps_taken']}")
        print(f"   Efficiency: {fastest['metrics']['efficiency']:.4f}")
    
    if failed:
        print(f"\n❌ Failed strategies:")
        for f in failed:
            print(f"   {f['name']}: reached {f['metrics']['final_odor']:.4f} odor level")
    
    return results

# Run the sensitivity analysis
analysis_results = parameter_sensitivity_analysis()

## Interactive Algorithm Comparison

Let's create an interactive widget that allows you to quickly compare different algorithm configurations side by side.

In [ ]:
# Create preset configurations for comparison
ALGORITHM_PRESETS = {
    'Standard 8-Direction': {
        'directions': [0, 45, 90, 135, 180, 225, 270, 315],
        'test_distance': 2.0,
        'base_speed': 0.3,
        'speed_multiplier': 2.0,
        'description': 'Balanced approach with 8 evenly spaced test directions'
    },
    'Simple Cardinal': {
        'directions': [0, 90, 180, 270],
        'test_distance': 2.0,
        'base_speed': 0.3,
        'speed_multiplier': 2.0,
        'description': 'Simple 4-direction approach (N, E, S, W)'
    },
    'High Precision': {
        'directions': [i * 22.5 for i in range(16)],
        'test_distance': 2.0,
        'base_speed': 0.2,
        'speed_multiplier': 1.5,
        'description': 'High precision with 16 test directions'
    },
    'Aggressive Explorer': {
        'directions': [0, 45, 90, 135, 180, 225, 270, 315],
        'test_distance': 3.0,
        'base_speed': 0.5,
        'speed_multiplier': 3.0,
        'description': 'Fast and aggressive with long-range sensing'
    },
    'Conservative': {
        'directions': [0, 60, 120, 180, 240, 300],
        'test_distance': 1.5,
        'base_speed': 0.2,
        'speed_multiplier': 1.0,
        'description': 'Slow and steady with shorter sensing range'
    }
}

def create_comparison_widget():
    """Create interactive widget for algorithm comparison."""
    
    # Create preset selector
    preset_selector = widgets.Dropdown(
        options=list(ALGORITHM_PRESETS.keys()),
        value='Standard 8-Direction',
        description='Algorithm Preset:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    # Create comparison button
    compare_button = widgets.Button(
        description='Run Comparison',
        button_style='success',
        layout=widgets.Layout(width='150px')
    )
    
    # Create output area
    output_area = widgets.Output()
    
    def run_comparison(button):
        """Run comparison between current settings and selected preset."""
        with output_area:
            clear_output(wait=True)
            
            preset_name = preset_selector.value
            preset_config = ALGORITHM_PRESETS[preset_name]
            
            print(f"🔄 Comparing Current Settings vs {preset_name}")
            print("" + "="*60)
            
            # Save current widget settings
            current_config = {
                'directions': list(params.directions_widget.value),
                'test_distance': params.distance_widget.value,
                'base_speed': params.base_speed_widget.value,
                'speed_multiplier': params.speed_mult_widget.value
            }
            
            # Run both configurations
            configs = [
                ('Current Settings', current_config),
                (preset_name, preset_config)
            ]
            
            results = []
            
            for config_name, config in configs:
                print(f"\n🧪 Testing {config_name}:")
                print(f"   Directions: {len(config['directions'])} ({config['directions'][:4]}...)")
                print(f"   Test distance: {config['test_distance']:.1f}")
                print(f"   Base speed: {config['base_speed']:.2f}")
                print(f"   Speed multiplier: {config['speed_multiplier']:.1f}")
                
                # Apply configuration
                params.directions_widget.value = config['directions']
                params.distance_widget.value = config['test_distance']
                params.base_speed_widget.value = config['base_speed']
                params.speed_mult_widget.value = config['speed_multiplier']
                
                # Run simulation
                with seed_context(EXPERIMENT_SEED):
                    sim_result = run_complete_simulation(max_steps=40, show_animation=False)
                
                results.append((config_name, sim_result))
                
                # Print results
                m = sim_result['metrics']
                print(f"   Result: {'✅ Success' if m['success'] else '❌ Failed'}")
                print(f"   Steps: {m['steps_taken']}, Distance: {m['total_distance']:.2f}")
                print(f"   Final odor: {m['final_odor']:.4f}, Efficiency: {m['efficiency']:.4f}")
            
            # Create comparison visualization
            fig, axes = plt.subplots(1, 3, figsize=(18, 6))
            
            # Plot trajectories
            ax1 = axes[0]
            im = ax1.imshow(odor_field, origin='lower', cmap='viridis', alpha=0.6,
                           extent=[0, odor_field.shape[1], 0, odor_field.shape[0]])
            
            colors = ['red', 'blue']
            for i, (name, result) in enumerate(results):
                trajectory = result['trajectory']
                ax1.plot(trajectory[:, 0], trajectory[:, 1], 
                        color=colors[i], linewidth=2, alpha=0.8, label=name)
                ax1.scatter(trajectory[0, 0], trajectory[0, 1], 
                           color=colors[i], s=100, marker='o', edgecolors='white')
                ax1.scatter(trajectory[-1, 0], trajectory[-1, 1], 
                           color=colors[i], s=150, marker='*', edgecolors='white')
            
            ax1.set_title('Trajectory Comparison', fontweight='bold')
            ax1.set_xlabel('X Position')
            ax1.set_ylabel('Y Position')
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            
            # Plot odor progression
            ax2 = axes[1]
            for i, (name, result) in enumerate(results):
                odor_readings = result['odor_readings']
                ax2.plot(odor_readings, color=colors[i], linewidth=2, 
                        marker='o', markersize=3, label=name)
            
            ax2.axhline(y=0.9, color='black', linestyle='--', alpha=0.7, 
                       label='Convergence Threshold')
            ax2.set_title('Odor Concentration Progress', fontweight='bold')
            ax2.set_xlabel('Step Number')
            ax2.set_ylabel('Odor Concentration')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            
            # Plot performance metrics
            ax3 = axes[2]
            metrics_names = ['Steps', 'Distance', 'Final Odor', 'Efficiency']
            
            x = np.arange(len(metrics_names))
            width = 0.35
            
            for i, (name, result) in enumerate(results):
                m = result['metrics']
                values = [m['steps_taken'], m['total_distance'], 
                         m['final_odor'], m['efficiency']]
                
                # Normalize for comparison (except final_odor which is already 0-1)
                normalized_values = [
                    values[0] / 50,  # steps (normalize to max 50)
                    values[1] / 100, # distance (normalize to max 100)
                    values[2],       # final_odor (already 0-1)
                    values[3] * 20   # efficiency (scale up for visibility)
                ]
                
                ax3.bar(x + i*width, normalized_values, width, 
                       color=colors[i], alpha=0.7, label=name)
            
            ax3.set_title('Performance Comparison (Normalized)', fontweight='bold')
            ax3.set_ylabel('Normalized Value')
            ax3.set_xticks(x + width/2)
            ax3.set_xticklabels(metrics_names)
            ax3.legend()
            ax3.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # Restore current settings
            params.directions_widget.value = current_config['directions']
            params.distance_widget.value = current_config['test_distance']
            params.base_speed_widget.value = current_config['base_speed']
            params.speed_mult_widget.value = current_config['speed_multiplier']
            
            # Print final comparison
            print("\n🏁 Comparison Summary:")
            print("" + "-"*40)
            
            current_metrics = results[0][1]['metrics']
            preset_metrics = results[1][1]['metrics']
            
            if current_metrics['success'] and preset_metrics['success']:
                if current_metrics['efficiency'] > preset_metrics['efficiency']:
                    print(f"🏆 Current settings are more efficient!")
                elif preset_metrics['efficiency'] > current_metrics['efficiency']:
                    print(f"🏆 {preset_name} is more efficient!")
                else:
                    print(f"🤝 Both configurations perform similarly")
            elif current_metrics['success']:
                print(f"🏆 Current settings succeed, {preset_name} fails")
            elif preset_metrics['success']:
                print(f"🏆 {preset_name} succeeds, current settings fail")
            else:
                print(f"⚠️ Both configurations fail to reach the target")
    
    compare_button.on_click(run_comparison)
    
    # Display preset description
    def show_preset_description(change):
        preset = ALGORITHM_PRESETS[change['new']]
        print(f"📝 {change['new']}: {preset['description']}")
    
    preset_selector.observe(show_preset_description, names='value')
    
    # Create layout
    control_box = widgets.HBox([
        preset_selector,
        compare_button
    ])
    
    display(widgets.VBox([
        widgets.HTML("<h3>🔬 Algorithm Comparison Tool</h3>"),
        widgets.HTML("<p>Select a preset algorithm to compare with your current settings:</p>"),
        control_box,
        output_area
    ]))
    
    # Show initial description
    initial_preset = ALGORITHM_PRESETS[preset_selector.value]
    print(f"📝 {preset_selector.value}: {initial_preset['description']}")

# Create the comparison widget
create_comparison_widget()

## Summary and Key Insights

This notebook has demonstrated the fundamental concepts of gradient-following algorithms for odor plume navigation. Let's summarize the key insights and learning outcomes.

In [ ]:
def generate_learning_summary():
    """Generate a comprehensive summary of learning insights."""
    
    print("📚 LEARNING SUMMARY: Odor Following Navigation Algorithms")
    print("" + "="*70)
    
    print("\n🎯 Key Algorithm Components:")
    print("" + "-"*30)
    print("1. Direction Testing: Systematic exploration of multiple directions")
    print("   • More directions = higher precision but slower execution")
    print("   • Cardinal directions (N,E,S,W) often sufficient for simple cases")
    print("   • 8-direction sensing provides good balance of speed and accuracy")
    
    print("\n2. Gradient Detection: Comparing odor concentrations")
    print("   • Local sampling reveals the steepest gradient direction")
    print("   • Test distance affects sensitivity vs. noise trade-off")
    print("   • Longer range sensing can help avoid local optima")
    
    print("\n3. Adaptive Speed Control: Gradient-based velocity adjustment")
    print("   • Base speed ensures continuous movement even in flat regions")
    print("   • Speed multiplier scales response to gradient strength")
    print("   • Speed limits prevent overshooting and instability")
    
    print("\n🔍 Performance Insights:")
    print("" + "-"*25)
    print("• Conservative approaches: Slower but more reliable")
    print("• Aggressive approaches: Faster but risk overshooting")
    print("• High-resolution sensing: Better precision at computational cost")
    print("• Simple cardinal directions: Often sufficient for well-defined gradients")
    
    print("\n🧠 Biological Relevance:")
    print("" + "-"*22)
    print("• Mimics natural chemotaxis behaviors in bacteria and insects")
    print("• Similar to how moths locate pheromone sources")
    print("• Comparable to olfactory navigation in marine animals")
    print("• Demonstrates importance of sensing range vs. movement speed")
    
    print("\n⚙️ Engineering Applications:")
    print("" + "-"*27)
    print("• Autonomous vehicle navigation in chemical environments")
    print("• Drone-based environmental monitoring and source localization")
    print("• Robotic systems for hazardous material detection")
    print("• Search and rescue operations in contaminated areas")
    
    print("\n📊 Parameter Guidelines:")
    print("" + "-"*22)
    print("• Start with 8-direction sensing for balanced performance")
    print("• Use test distance of 1-3 times your movement step size")
    print("• Set base speed to 10-30% of maximum speed")
    print("• Configure speed multiplier between 1-3 for stability")
    print("• Adjust convergence threshold based on required precision")
    
    print("\n🔬 Experimental Reproducibility:")
    print("" + "-"*32)
    print(f"• Experiment ID: {EXPERIMENT_ID}")
    print(f"• Random seed: {EXPERIMENT_SEED}")
    print(f"• Environment: {ENV_WIDTH}×{ENV_HEIGHT} Gaussian odor field")
    print(f"• Platform: {seed_manager.get_reproducibility_info()['platform_info']['platform']}")
    
    print("\n🚀 Next Steps for Further Learning:")
    print("" + "-"*35)
    print("1. Experiment with different odor field configurations")
    print("2. Implement more sophisticated navigation strategies")
    print("3. Add obstacles and complex environments")
    print("4. Compare with other search algorithms (e.g., random walk)")
    print("5. Explore multi-agent coordination and swarm behaviors")
    print("6. Investigate real-world sensor noise and uncertainty")
    
    print("\n✨ Congratulations!")
    print("You've successfully explored the fundamentals of gradient-following")
    print("navigation algorithms. These concepts form the basis for many")
    print("real-world autonomous navigation systems.")

# Generate the learning summary
generate_learning_summary()

## Export Results and Configuration

Finally, let's save our experimental configuration and results for future reference.

In [ ]:
import json
from datetime import datetime

# Prepare export data
export_data = {
    'metadata': {
        'experiment_id': EXPERIMENT_ID,
        'timestamp': datetime.now().isoformat(),
        'notebook_version': '02_odor_following_demo_v1.0',
        'random_seed': EXPERIMENT_SEED
    },
    'environment': {
        'width': ENV_WIDTH,
        'height': ENV_HEIGHT,
        'odor_field_stats': {
            'min': float(np.min(odor_field)),
            'max': float(np.max(odor_field)),
            'mean': float(np.mean(odor_field)),
            'std': float(np.std(odor_field))
        }
    },
    'final_parameters': {
        'test_directions': list(params.directions_widget.value),
        'test_distance': params.distance_widget.value,
        'base_speed': params.base_speed_widget.value,
        'speed_multiplier': params.speed_mult_widget.value,
        'convergence_threshold': params.threshold_widget.value
    },
    'reproducibility_info': seed_manager.get_reproducibility_info()
}

# Display export information
print("💾 Experiment Configuration Export")
print("" + "="*40)
print(f"Experiment ID: {export_data['metadata']['experiment_id']}")
print(f"Timestamp: {export_data['metadata']['timestamp']}")
print(f"Random Seed: {export_data['metadata']['random_seed']}")
print("\nFinal Algorithm Parameters:")
for key, value in export_data['final_parameters'].items():
    print(f"  {key}: {value}")

# Create JSON export (commented out - uncomment to save to file)
# export_filename = f"odor_following_experiment_{EXPERIMENT_ID}.json"
# with open(export_filename, 'w') as f:
#     json.dump(export_data, f, indent=2)
# print(f"\n✅ Configuration saved to: {export_filename}")

print("\n🎉 Notebook completed successfully!")
print("Feel free to modify parameters and re-run sections to explore different behaviors.")